# <div style="padding: 25px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFA500"><b><span style='color:#FFA500'></span></b> <b>1. Initial Setting</b></div>

<div style="background-color:; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
    💡<b>What's New</b><br>
     Adding Puncutal Features & Ratio Features & Mid,SuperLong Sentence, Word, Parargraph Features</div>

In [ ]:
import os
import gc 
import ctypes
import random
import time
import string
import re
from tqdm import tqdm
import pickle

import pandas as pd, numpy as np
import polars as pl # For Feature Engineering

import matplotlib.pyplot as plt
import seaborn as sns 

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import VotingRegressor

os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

import warnings 
warnings.filterwarnings('ignore')

#### <b><span style='color:#FFA500'> | </span> Configure</b>

In [ ]:
class CFG:
    SEED = 2024
    VER = 3
    LOAD_MODELS_FROM = None
    LOAD_FEATURES_FROM = None
    BASE_PATH = '/kaggle/input/learning-agency-lab-automated-essay-scoring-2/'

#### <b><span style='color:#FFA500'> | </span> Clean Memory</b>

In [ ]:
Clean = True

def clean_memory():
    if Clean:
        ctypes.CDLL('libc.so.6').malloc_trim(0)
        gc.collect()
        
clean_memory()        

#### <b><span style='color:#FFA500'> | </span> Seed Everything</b>

In [ ]:
def seed_everything(): # To proudce simliar result in each run 
    random.seed(CFG.SEED)
    np.random.seed(CFG.SEED)
    os.environ['PYTHONHASHSEED'] = str(CFG.SEED)
    
seed_everything()

# <div style="padding: 25px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFA500"><b><span style='color:#FFA500'></span></b> <b>2. Road and Read Data</b></div>

#### <b><span style='color:#FFA500'> | </span> Meta Data</b>


- **`[train/test].csv`**
  - `essay_id`: The unique ID of the essay
  - `full_text`: The full essay reponse
  - `score`: Holistic score of the essay on a 1-6 scale
  
  
- **`sample_submission.csv`**
  - `essay_id`: The unique ID of the essay
  - `score`: The predicted holistic score of the essay on a 1-6 scale
  
  
> [Link to the Holistic Scoring Rubric](https://storage.googleapis.com/kaggle-forum-message-attachments/2733927/20538/Rubric_%20Holistic%20Essay%20Scoring.pdf)

#### <b><span style='color:#FFA500'> | </span> Train Data</b>

In [ ]:
df_train = pd.read_csv(CFG.BASE_PATH + 'train.csv')

print('Shape of Train: ', df_train.shape)
print(display(df_train.head()))

In [ ]:
plt.figure(figsize=(12,6))
sns.countplot(x=df_train['score'])
plt.title('Distribution of Score')
plt.xlabel('Score of Essay')
plt.ylabel('Frequency')
plt.show()

#### <b><span style='color:#FFA500'> | </span> Test Data</b>

In [ ]:
df_test = pd.read_csv(CFG.BASE_PATH + 'test.csv')

print('Shape of Test: ', df_test.shape)
print(display(df_test.head()))

In [ ]:
# os.path.join(CFG.BASE_PATH,'train.csv')
# CFG.BASE_PATH + 'train.csv'

train = pl.read_csv(os.path.join(CFG.BASE_PATH,'train.csv')).with_columns(
    pl.col('full_text').str.split(by='\n\n').alias('paragraph'))
test = pl.read_csv(os.path.join(CFG.BASE_PATH,'test.csv')).with_columns(
    pl.col('full_text').str.split(by='\n\n').alias('paragraph')
)

schema_train = train.schema # MetaData
schema_test = test.schema # MetaData

# <div style="padding: 25px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFA500"><b><span style='color:#FFA500'></span></b> <b>3. FeatureGenerator</b></div>

<div style="background-color:; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌<b>Features & TFIDF    
   
    
    Thanks for Sharing! @yell725</b>

https://www.kaggle.com/code/ye11725/tfidf-lgbm-baseline-with-code-comments
</div>

In [ ]:
def removeHTML(x):
    html = re.compile(r'<.*?>')
    return html.sub(r'',x) # html -> ''

def dataPreprocessing(x):
    
    x = x.lower()
    # remove html
    x = removeHTML(x)
    
    #remove string with starting @
    x = re.sub("@\w+",'',x)
    
    # remove number
    x = re.sub("'\d+",'',x)
    x = re.sub("\d+",'',x)
    
    # remove url
    x = re.sub("http\w+",'',x)
    
    # Replace consecutive empty spaces with a single empty spaces
    x = re.sub(r'\s+', " ", x)
    
    # Replace consecutive commas and periods with a single comma and period
    x = re.sub(r'\.+', ".",x)
    x = re.sub(r'\,+', ".",x)
    # Remove empty character at the beginning and end
    x = x.strip()
    return x

#### <b><span style='color:#FFA500'> | </span> Paragraph</b>

In [ ]:
paragraph_features = ['paragraph_len','paragraph_sentence_cnt','paragraph_word_cnt']

def Paragraph_Features(x):
    # Expand the paragraph list to several lines of data
    x = x.explode('paragraph') 
    
    print('Paragraph Preprocessing')
    x = x.with_columns(
         pl.col('paragraph').map_elements(dataPreprocessing)
    )
    
    print('Caculate the length of each paragraph')
    x = x.with_columns(
         pl.col('paragraph').map_elements(lambda x: len(x)).alias("paragraph_len")
    )
    print('Caculate the number of sentences and words in each paragraph')
    x = x.with_columns(
         pl.col('paragraph').map_elements(lambda x: len(x.split('.'))).alias('paragraph_sentence_cnt'),
         pl.col('paragraph').map_elements(lambda x: len(x.split(' '))).alias('paragraph_word_cnt')
    )
    return x

def Paragraph_aggregation(x):    
    
    print('Aggregation')
    aggs = [
    *[pl.col('paragraph').filter(pl.col('paragraph_len') >= i).count().alias(f'paragraph_{i}_cnt') for i in [100,150, 200, 250, 300, 350, 400, 450, 500, 600, 800]],
    *[pl.col('paragraph').filter(pl.col('paragraph_len') <= i).count().alias(f'paragraph_{i}_cnt_v2') for i in [100,200]],
    *[pl.col('paragraph').filter(pl.col('paragraph_sentence_cnt') >= i).count().alias(f'paragraph_sentence_{i}_cnt') for i in [2,4,6,8,10]],
    *[pl.col('paragraph').filter((pl.col('paragraph_len') <= 300) & (pl.col('paragraph_len') > 100)).count().alias(f'short_paragraph_cnt')],    
    *[pl.col('paragraph').filter((pl.col('paragraph_len') <= 500) & (pl.col('paragraph_len') > 300)).count().alias(f'mid_paragraph_cnt')],
    *[pl.col('paragraph').filter((pl.col('paragraph_len') <= 700) & (pl.col('paragraph_len') > 500)).count().alias(f'long_paragraph_cnt')],
    *[pl.col('paragraph').filter((pl.col('paragraph_sentence_cnt') <= 4) & (pl.col('paragraph_sentence_cnt') > 2)).count().alias(f'short_paragraph_sentence_cnt')],
    *[pl.col('paragraph').filter((pl.col('paragraph_sentence_cnt') <= 8) & (pl.col('paragraph_sentence_cnt') > 4)).count().alias(f'mid_paragraph_sentence_cnt')],
    *[pl.col('paragraph').filter((pl.col('paragraph_sentence_cnt') <= 10) & (pl.col('paragraph_sentence_cnt') > 8)).count().alias(f'long_paragraph_sentence_cnt')],    
    *[pl.col('paragraph').filter(pl.col('paragraph_word_cnt') >= i).count().alias(f'paragraph_word_{i}_cnt') for i in [30,60,90,120]],
    *[pl.col('paragraph').filter((pl.col('paragraph_word_cnt') <= 60) & (pl.col('paragraph_word_cnt') > 30)).count().alias(f'short_paragraph_word_cnt')],
    *[pl.col('paragraph').filter((pl.col('paragraph_word_cnt') <= 90) & (pl.col('paragraph_word_cnt') > 60)).count().alias(f'mid_paragraph_word_cnt')],
    *[pl.col('paragraph').filter((pl.col('paragraph_word_cnt') <= 120) & (pl.col('paragraph_word_cnt') > 90)).count().alias(f'long_paragraph_word_cnt')],  
    *[pl.col('paragraph').count().alias('paragraph_cnt')],   
    ]
      
    df = x.group_by(['essay_id'], maintain_order=True).agg(aggs).sort("essay_id")
    df = df.to_pandas() # polars -> pandas
    
    return df

#### <b><span style='color:#FFA500'> | </span> Sentence</b>

In [ ]:
sentence_features = ['sentence_len','sentence_word_cnt','sentence_len_space_ratio','sentence_word_space_ratio']

def Sentence_Features(x):
    print('Preprocess full_text and use periods to segment sentences in the text')
    x = x.with_columns(
        pl.col('full_text').map_elements(lambda x: dataPreprocessing(x)).str.split(".").alias('sentence')
    )
    x = x.explode('sentence')
    
    print('Caculate the length of a sentence') 
    x = x.with_columns(
        pl.col('sentence').map_elements(lambda x: x.count(' ')).alias('sentence_space_cnt'))
    x = x.filter(pl.col('sentence_space_cnt')>0)
    x = x.with_columns(
        pl.col('sentence').map_elements(lambda x: len(x)).alias('sentence_len'))
    x = x.filter(pl.col('sentence_len') > 3)
    x = x.with_columns(
        (pl.col('sentence_len')/pl.col('sentence_space_cnt')).alias('sentence_len_space_ratio'))
        
    print('Count the number of words in each sentence')
    x = x.with_columns(
        pl.col('sentence').map_elements(lambda x: len(x.split(" "))).alias("sentence_word_cnt"))
    x = x.with_columns(
        (pl.col('sentence_word_cnt')/pl.col('sentence_space_cnt')).alias('sentence_word_space_ratio'))
     
    return x

def Sentence_aggregation(x):    
    
    print('Aggregation')
    aggs = [
    *[pl.col('sentence').filter(pl.col('sentence_len') >= i).count().alias(f'sentence_{i}_cnt') for i in [30,40,50,60,70,80,100,150]],    
    *[pl.col('sentence').filter(pl.col('sentence_len') <= i).count().alias(f'sentence_{i}_cnt_v2') for i in [10,20,30]],
    *[pl.col('sentence').filter((pl.col('sentence_len') <= 50) & (pl.col('sentence_len') > 30)).count().alias(f'short_sentence_cnt')],    
    *[pl.col('sentence').filter((pl.col('sentence_len') <= 70) & (pl.col('sentence_len') > 50)).count().alias(f'mid_sentence_cnt')], 
    *[pl.col('sentence').filter((pl.col('sentence_len') <= 100) & (pl.col('sentence_len') > 70)).count().alias(f'long_sentence_cnt')],
    *[pl.col('sentence').filter(pl.col('sentence_word_cnt') >= i).count().alias(f'sentence_word_{i}_cnt') for i in [5,10,15,20]], 
    *[pl.col('sentence').filter((pl.col('sentence_word_cnt') <= 10) & (pl.col('sentence_word_cnt') > 5)).count().alias(f'short_sentence_word_cnt')],    
    *[pl.col('sentence').filter((pl.col('sentence_word_cnt') <= 15) & (pl.col('sentence_word_cnt') > 10)).count().alias(f'mid_sentence_word_cnt')], 
    *[pl.col('sentence').filter((pl.col('sentence_word_cnt') <= 20) & (pl.col('sentence_word_cnt') > 15)).count().alias(f'long_sentence_word_cnt')],    
    *[pl.col('sentence').count().alias('sentence_cnt')],
    *[pl.col(feat).max().alias(f'{feat}_max') for feat in sentence_features],
    *[pl.col(feat).mean().alias(f'{feat}_mean') for feat in sentence_features],
    *[pl.col(feat).min().alias(f'{feat}_min') for feat in sentence_features],
    *[pl.col(feat).std().alias(f'{feat}_std') for feat in sentence_features],
    *[pl.col(feat).sum().alias(f'{feat}_sum') for feat in sentence_features], 
    ]
    
    df = x.group_by(['essay_id'], maintain_order=True).agg(aggs).sort("essay_id")
    
    df = df.to_pandas() # polars -> pandas
    
    return df

#### <b><span style='color:#FFA500'> | </span> Word</b>

In [ ]:
word_features = ['word_len',]

def Word_Features(x):
    print('Preprocess full_text and use spaces to seperate words from the text')
    x = x.with_columns(
        pl.col('full_text').map_elements(lambda x: dataPreprocessing(x)).str.split(".").alias('sentence')
    )
    x = x.explode('sentence')
    
    x = x.with_columns(
        pl.col('sentence').map_elements(lambda x: dataPreprocessing(x)).str.split(" ").alias('word')
    )
    x = x.explode('word')
    
    print('Caculate the length of a word') 
    x = x.with_columns(
        pl.col('word').map_elements(lambda x: len(x)).alias('word_len'))
    x = x.filter(pl.col('word_len')>0)

    return x

def Word_aggregation(x):    
    
    print('Aggregation')
    aggs = [
    *[pl.col('word').filter(pl.col('word_len') >= i).count().alias(f'word_{i}_cnt') for i in [3,4,5,6,8,10,15]],
    *[pl.col('word').filter(pl.col('word_len') <= i).count().alias(f'word_{i}_cnt_v2') for i in [2,3]],
    *[pl.col('word').filter((pl.col('word_len') <= 4) & (pl.col('word_len') > 2)).count().alias(f'short_word_cnt')],
    *[pl.col('word').filter((pl.col('word_len') <= 7) & (pl.col('word_len') > 4)).count().alias(f'mid_word_cnt')], 
    *[pl.col('word').filter((pl.col('word_len') <= 10) & (pl.col('word_len') > 7)).count().alias(f'long_word_cnt')],     
    *[pl.col('word').count().alias('word_cnt')],   
    *[pl.col(feat).max().alias(f'{feat}_max') for feat in word_features],    
    *[pl.col(feat).mean().alias(f'{feat}_mean') for feat in word_features],
    *[pl.col(feat).min().alias(f'{feat}_min') for feat in word_features],
    *[pl.col(feat).std().alias(f'{feat}_std') for feat in word_features],
    *[pl.col(feat).sum().alias(f'{feat}_sum') for feat in word_features],
    *[pl.col(feat).quantile(0.25).alias(f'{feat}_q1') for feat in word_features],
    *[pl.col(feat).quantile(0.75).alias(f'{feat}_q3') for feat in word_features],   
    ]
    
    df = x.group_by(['essay_id'], maintain_order=True).agg(aggs).sort("essay_id")
    df = df.with_columns(
    *[(pl.col(f'word_{i}_cnt')/pl.col(f'word_2_cnt_v2')).alias(f'word_2_{i}_cnt_ratio') for i in [3,4,5,6,8,10,15]], 
    *[(pl.col(f'word_{i}_cnt')/pl.col(f'word_3_cnt_v2')).alias(f'word_3_{i}_cnt_ratio') for i in [3,4,5,6,8,10,15]], 
    *[(pl.col(f'short_word_cnt')/ pl.col(f'word_{i}_cnt_v2')).alias(f'short_word_ratio_{i}') for i in [2,3]], 
    *[(pl.col(f'mid_word_cnt')/ pl.col(f'word_{i}_cnt_v2')).alias(f'mid_word_ratio_{i}') for i in [2,3]], 
    *[(pl.col(f'long_word_cnt')/ pl.col(f'word_{i}_cnt_v2')).alias(f'long_word_ratio_{i}') for i in [2,3]],        
    
    ).sort("essay_id")
    df = df.to_pandas() # polars -> pandas
    
    return df

In [ ]:
def Punctual_Features(x):

    
    print("Caculate number of punctuation")
    x = x.explode('paragraph') 
    x = x.with_columns(
         pl.col('paragraph').map_elements(dataPreprocessing)
    )
    x = x.with_columns(
        pl.col('paragraph').map_elements(lambda x: x.count('?') + x.count('!')).alias('paragraph_punct_excl'),
        pl.col('paragraph').map_elements(lambda x: x.count('"') + x.count("'") + x.count('`')).alias('paragraph_punct_quotes'),
        pl.col('paragraph').map_elements(lambda x: x.count(',')).alias('paragraph_punct_para'),
    )
    
    return x

punctual_features = ['paragraph_punct_excl','paragraph_punct_quotes','paragraph_punct_para']

def Punctual_aggregation(x):    
    
    print('Aggregation')
    aggs = [
    *[pl.col(feat).count().alias(f'{feat}_cnt') for feat in punctual_features],   
    *[pl.col(feat).max().alias(f'{feat}_max') for feat in punctual_features],
    *[pl.col(feat).mean().alias(f'{feat}_mean') for feat in punctual_features],
    *[pl.col(feat).min().alias(f'{feat}_min') for feat in punctual_features],
    *[pl.col(feat).std().alias(f'{feat}_std') for feat in punctual_features],
    *[pl.col(feat).sum().alias(f'{feat}_sum') for feat in punctual_features],      
    ]
    
    df = x.group_by(['essay_id'], maintain_order=True).agg(aggs).sort("essay_id")
    df = df.to_pandas() # polars -> pandas
    
    return df

#### <b><span style='color:#FFA500'> | </span> TfidfVectorizer</b>

In [ ]:
vectorizer = TfidfVectorizer(
    tokenizer = lambda x: x,
    preprocessor = lambda x: x,
    token_pattern=None,
    strip_accents='unicode',
    analyzer= 'word',
    ngram_range = (1,4),
    min_df =0.05,
    max_df=0.95,
    sublinear_tf = True # Term Frequency Log Scaling 
    )

# TfidfVectorizer parameter

# Fit all datasets into TfidfVectorizer
train_tfid = vectorizer.fit_transform([i for i in train['full_text']])

              
# Convert to array
dense_matrix = train_tfid.toarray()

# Convert to dataframe
df = pd.DataFrame(dense_matrix)
df.columns = [f'tfidf_{i}' for i in range(len(df.columns))]


df['essay_id'] = df_train['essay_id']

#### <b><span style='color:#FFA500'> | </span> CountVectorizer</b>

<div style="background-color:; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌<b>CountVectorizer

Thanks for Sharing! *@yukiZ*

https://www.kaggle.com/code/ye11725/tfidf-lgbm-baseline-with-code-comments
</div>

In [ ]:
vectorizer_cnt = CountVectorizer(
      tokenizer=lambda x: x,
      preprocessor=lambda x: x,
      token_pattern=None,
      strip_accents='unicode',
      analyzer = 'word',
      ngram_range=(1,3),
      min_df=0.10,
      max_df=0.90,)
    
# TfidfVectorizer parameter

# Fit all datasets into TfidfVectorizer
train_cnt = vectorizer_cnt.fit_transform([i for i in train['full_text']])

              
# Convert to array
dense_matrix2 = train_cnt.toarray()

# Convert to dataframe
df2 = pd.DataFrame(dense_matrix2)
df2.columns = [f'cnt_{i}' for i in range(len(df2.columns))]


df2['essay_id'] = df_train['essay_id']

<div style="background-color:; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌<b>WordVec
   </div>

In [ ]:
from gensim.models import Word2Vec
from sklearn.decomposition import PCA

def W2v_train(x):
    print('training Word2Vec Model')
    w2v = Word2Vec(sentences=x['full_text'], vector_size=50, window=4, min_count=1, workers=4)
    return x, w2v

def W2v_Features(sentence, model):
    words = sentence.split()
    word_embeddings = [model.wv[word] for word in words if word in model.wv]

    if len(word_embeddings) > 0:
        return np.mean(word_embeddings, axis=0)
    else:
        return np.zeros(model.vector_size)


def W2v_aggregation(x,model):    
    print('Aggregation Embedding Features')
    x = x.with_columns(
        pl.col('full_text').apply(lambda sentence: W2v_Features(sentence, model)).alias('text_embedding')
    )
    feature_columns = [f'text_embedding_{i}' for i in range(model.vector_size)]

    for i, col_name in enumerate(feature_columns):
        x = x.with_columns(
            pl.col('text_embedding').apply(lambda embeddings: embeddings[i]).alias(col_name)
        )

    
    
    df = x.drop(['full_text','score','text_embedding','paragraph'])
    df = df.to_pandas()
    
    return df

<div style="background-color:; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌<b>Merging datasets
   </div>

In [ ]:
if CFG.LOAD_FEATURES_FROM is None: 
    
    train_feats1 = Paragraph_Features(train)
    train_feats1 = Paragraph_aggregation(train_feats1)
    train_feats2 = Sentence_Features(train)
    train_feats2 = Sentence_aggregation(train_feats2)
    train_feats3 = Word_Features(train)
    train_feats3 = Word_aggregation(train_feats3)
    #train_feats4 = Punctual_Features(train)
    #train_feats4 = Punctual_aggregation(train_feats4)
    
    train_feats = train_feats1.merge(train_feats2, on='essay_id', how='left')
    train_feats = train_feats.merge(train_feats3, on='essay_id', how='left')
    #train_feats = train_feats.merge(train_feats4, on='essay_id', how='left')
    train_feats = train_feats.merge(df, on='essay_id', how='left')
    train_feats = train_feats.merge(df2, on='essay_id', how='left')
    train_feats['score'] = df_train['score'].values
else:
    None

In [ ]:
if CFG.LOAD_FEATURES_FROM is None: 
    print('Save train_feats.csv')
    train_feats.to_csv(f'train_feats_{CFG.VER}.csv', index=False)
else: 
    print('Load train_feats.csv')
    train_feats = pd.read_csv(CFG.LOAD_FEATURES_FROM)

In [ ]:
display(train_feats.head())

# <div style="padding: 25px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFA500"><b><span style='color:#FFA500'></span></b> <b>4. LightGBM Model</b></div>

In [ ]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import cohen_kappa_score

import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation
print('LightGBM Version: ', lgb.__version__)

#### <b><span style='color:#FFA500'> | </span> Quadatric Weighted Kappa</b>

In [ ]:
def quadratic_weighted_kappa(y_true, y_pred):
    y_true = y_true + a
    y_pred = (y_pred + a).clip(1, 6).round()
    qwk = cohen_kappa_score(y_true, y_pred, weights="quadratic")
    return 'QWK', qwk, True
def qwk_obj(y_true, y_pred):
    labels = y_true + a
    preds = y_pred + a
    preds = preds.clip(1, 6)
    f = 1/2*np.sum((preds-labels)**2)
    g = 1/2*np.sum((preds-a)**2+b)
    df = preds - labels
    dg = preds - a
    grad = (df/g - f*dg/g**2)*len(labels)
    hess = np.ones(len(labels))
    return grad, hess
a = 2.948
b = 1.092

#### <b><span style='color:#FFA500'> | </span> LightGBM Pipeline</b>

In [ ]:
categorical_columns = train_feats.select_dtypes(include=['object','category']).columns.tolist()
FEATURES = [col for col in train_feats.columns if col not in categorical_columns + ['score']]
TARGET = 'score'

<div style="background-color:; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌<b> Optuna(Cross Validation: train_test_split)
    </div>

In [ ]:
# from sklearn.model_selection import train_test_split
# import optuna

In [ ]:
'''
def lgb_objective(trial):


    params = {    
              'objective': qwk_obj,
              'learning_rate': trial.suggest_float('learning_rate', 0.001,0.5),
              'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1,1.0),
              'max_depth': trial.suggest_int('max_depth', 4,32),
              'num_leaves': trial.suggest_int('num_leaves', 10,50),
              'reg_alpha': trial.suggest_float('reg_alpha', 0.1,1.0),
              'reg_lambda': trial.suggest_float('reg_lambda', 0.1,1.0),
              'n_estimators': trial.suggest_int('n_estimators', 200,1500),
    }
    
    train_x, valid_x, train_y, valid_y = train_test_split(train_feats[FEATURES], train_feats[TARGET], test_size=0.2, random_state=CFG.SEED)
    model = lgb.LGBMRegressor(**params)
    
    train_y = train_y - a
         
    valid_y = valid_y - a
    
    model.fit(train_x, train_y,
                  eval_set = [(valid_x, valid_y)],
                  eval_metric = quadratic_weighted_kappa,
                  callbacks = [early_stopping(stopping_rounds=100)]
               )
    oof = model.predict(valid_x, num_iteration=model.best_iteration_)    
    cv = cohen_kappa_score(valid_y + a, (oof+a).clip(1,6).round(), weights="quadratic")

    
    return -cv
'''

In [ ]:
#study = optuna.create_study(direction='minimize', study_name='Regression') 
#study.optimize(lgb_objective, n_trials=20, show_progress_bar=True)

In [ ]:
# print(f'Best Trial: score{study.best_value}, param{study.best_params}')

In [ ]:
# optuna.visualization.plot_parallel_coordinate(study)

<div style="background-color:; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌<b> Optuna Plot
    </div>

![](https://www.googleapis.com/download/storage/v1/b/kaggle-forum-message-attachments/o/inbox%2F16438831%2F11ea073c666ee5e4a1e08c75805c4f90%2Fplot.JPG?generation=1713440057876069&alt=media)

In [ ]:
def lightgbm():
    all_oof = []
    all_true = []
    
    skf = StratifiedKFold(n_splits=5, random_state=CFG.SEED, shuffle=True)
    for i, (train_index, valid_index) in enumerate(skf.split(train_feats, train_feats[TARGET])):
      
        print('#'*25)
        print(f'### Fold {i+1}')
        print(f'### train size {len(train_index)}, valid size {len(valid_index)}')
        print('#'*25)
                              
        model = lgb.LGBMRegressor(
                objective = qwk_obj,
                metrics = 'None',
                learning_rate = 0.10,
                colsample_bytree = 0.8, 
                max_depth = 25, 
                num_leaves = 20, 
                reg_alpha = 0.9,
                reg_lambda = 0.2,
                n_estimators = 1024,
                class_weight='balanced',
                random_state=CFG.SEED,
                verbosity = - 1)
                                               
        train_x = np.clip(train_feats.loc[train_index, FEATURES].fillna(0),
                          0, 10000)
        train_y = train_feats.loc[train_index, TARGET] - a
                                               
        valid_x = np.clip(train_feats.loc[valid_index, FEATURES].fillna(0),
                          0,10000)
        valid_y = train_feats.loc[valid_index, TARGET] - a
    
        model.fit(train_x, train_y,
                  eval_set = [(valid_x, valid_y)],
                  eval_metric = quadratic_weighted_kappa,
                  callbacks = [early_stopping(stopping_rounds=100)]
               )
    
        # Save Model
        pickle.dump(model, open(f'LGB_v{CFG.VER}_f{i}.pkl', 'wb'))
      
        oof = model.predict(valid_x, num_iteration=model.best_iteration_)
        all_oof.append(oof + a)
        all_true.append(valid_y.values + a)
      
        del train_x,train_y,valid_x,valid_y, oof, model
        clean_memory()
                                           
    all_oof = np.concatenate(all_oof)
    all_true = np.concatenate(all_true)

    oof = pd.DataFrame(all_oof.copy())
    oof['id'] = np.arange(len(oof))

    true = pd.DataFrame(all_true.copy())
    true['id'] = np.arange(len(true))

    cv = cohen_kappa_score(true[0], oof[0].clip(1,6).round(), weights="quadratic")
    print('CV Score for LightGBM = ',cv)

In [ ]:
if CFG.LOAD_MODELS_FROM is None:
    print('Training LightGBM')
    lightgbm()
else: 
    None

<div style="background-color:; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌<b>Feature Importance
    </div>

In [ ]:
if CFG.LOAD_MODELS_FROM:
    model = pickle.load(open(f'{CFG.LOAD_MODELS_FROM}LGB_v{CFG.VER}_f0.pkl', 'rb'))
else: 
    model = pickle.load(open(f'LGB_v{CFG.VER}_f0.pkl', 'rb'))

df_importance = pd.DataFrame({
        'features_name': FEATURES,
        'importance': model.feature_importances_,
    })
df_importance = df_importance.sort_values(by='importance', ascending=False)

plt.figure(figsize=(12,6))
plt.bar(data=df_importance.head(30), x='features_name', height='importance', color='pink', edgecolor='black')
plt.title('Distribution of Feature Importance of LightGBM')
plt.xticks(rotation=90)
plt.show()

# <div style="padding: 25px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#FFA500"><b><span style='color:#FFA500'></span></b> <b>5. Inference</b></div>

#### <b><span style='color:#FFA500'> | </span> Submit to Kaggle</b>

In [ ]:
# del model, train; gc.collect()

In [ ]:
test_tfid = vectorizer.transform([i for i in test['full_text']])
dense_matrix = test_tfid.toarray()
df3 = pd.DataFrame(dense_matrix)
tfid_columns = [ f'tfidf_{i}' for i in range(len(df3.columns))]
df3.columns = tfid_columns
df3['essay_id'] = df_test['essay_id']

In [ ]:
test_cnt = vectorizer_cnt.transform([i for i in test['full_text']])
dense_matrix = test_cnt.toarray()
df4 = pd.DataFrame(dense_matrix)
cnt_columns = [ f'cnt_{i}' for i in range(len(df4.columns))]
df4.columns = cnt_columns
df4['essay_id'] = df_test['essay_id']

<div style="background-color:; color:#19180F; font-size:15px; font-family:Verdana; padding:10px; border: 2px solid #19180F; border-radius:10px"> 
📌<b>Merging test datasets
    </div>

In [ ]:
%%time 
test_feats1 = Paragraph_Features(test)
test_feats1 = Paragraph_aggregation(test_feats1)
test_feats2 = Sentence_Features(test)
test_feats2 = Sentence_aggregation(test_feats2)
test_feats3 = Word_Features(test)
test_feats3 = Word_aggregation(test_feats3)
#test_feats4 = Punctual_Features(test)
#test_feats4 = Punctual_aggregation(test_feats4)

In [ ]:
test_feats = test_feats1.merge(test_feats2, on='essay_id', how='left')
test_feats = test_feats.merge(test_feats3, on='essay_id', how='left')
#test_feats = test_feats.merge(test_feats4, on='essay_id', how='left')
test_feats = test_feats.merge(df3, on='essay_id', how='left')
test_feats = test_feats.merge(df4, on='essay_id', how='left')
print('Shape of test_feats:', test_feats.shape)
display(test_feats.head())

In [ ]:
preds = []
categorical_columns = test_feats.select_dtypes(include=['object','category']).columns.tolist()
FEATURES = [col for col in test_feats.columns if col not in categorical_columns]


for i in range(5):
    print(f'Fold {i+1}')
    if CFG.LOAD_MODELS_FROM:
        model = pickle.load(open(f'{CFG.LOAD_MODELS_FROM}LGB_v{CFG.VER}_f{i}.pkl', 'rb'))
    else: 
        model = pickle.load(open(f'LGB_v{CFG.VER}_f{i}.pkl', 'rb'))
        
    pred = model.predict(test_feats[FEATURES]) + a
    preds.append(pred)
pred1 = np.mean(preds,axis=0)          
   

In [ ]:
sub = pd.DataFrame({'essay_id': df_test.essay_id.values})
sub['score'] = pred1.clip(1,6).round()
sub.to_csv('submission.csv',index=False)
print('Submission shape', sub.shape)
sub.head()